# Lab 1

## Ex.1

In [2]:
import time
print("hello")

hello


In [3]:
import time
import board
import adafruit_dht
import psutil

import datetime 

import pandas as pd

# We first check if a libgpiod process is running. If yes, we kill it!
for proc in psutil.process_iter():
    if proc.name() == 'libgpiod_pulsein' or proc.name() == 'libgpiod_pulsei':
        proc.kill()

sensor = adafruit_dht.DHT11(board.D4)

frequency = 6
period = 120

dates = []
hours = []
hums = []
temps = []

count = 0

while count <= int(period/frequency):
    try:
        temp = sensor.temperature
        humidity = sensor.humidity
        date = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        dates.append(date.split(" ")[0])
        hours.append(date.split(" ")[1])

        temps.append(temp)
        hums.append(humidity)

        count += 1

    except RuntimeError as error:
        print(error.args[0])
        
        continue

    time.sleep(frequency)


df = pd.DataFrame({'dates':dates,'hours':hours, 'temps':temps, 'hums':hums})
print(df)
df.to_csv('dataset_ex1.csv', index=False, header=False)



Checksum did not validate. Try again.
         dates     hours  temps  hums
0   04/11/2021  17:16:32     20    40
1   04/11/2021  17:16:39     20    41
2   04/11/2021  17:16:45     21    40
3   04/11/2021  17:16:51     21    40
4   04/11/2021  17:16:57     21    40
5   04/11/2021  17:17:04     21    40
6   04/11/2021  17:17:10     21    40
7   04/11/2021  17:17:16     21    39
8   04/11/2021  17:17:22     21    39
9   04/11/2021  17:17:29     21    39
10  04/11/2021  17:17:35     21    39
11  04/11/2021  17:17:41     21    39
12  04/11/2021  17:17:47     21    39
13  04/11/2021  17:17:54     21    39
14  04/11/2021  17:18:00     21    39
15  04/11/2021  17:18:06     21    39
16  04/11/2021  17:18:12     21    39
17  04/11/2021  17:18:19     21    39
18  04/11/2021  17:18:25     21    39
19  04/11/2021  17:18:31     21    39
20  04/11/2021  17:18:38     21    39


## Ex.2

In [ ]:
# Use the pyaudio package to drive the microphone 
# (with the blocking mode) and the wave package to 
# store the samples on disk.

import pyaudio
import wave

import os

from argparse import ArgumentParser


def record_audio(args):

    # define pyaudio object
    p = pyaudio.PyAudio()

    # define the format
    if args.format == 'Int8':
        format = pyaudio.paInt8
    elif args.format == 'Int16':
        format = pyaudio.paInt16
    elif args.format == 'Int32':
        format = pyaudio.paInt32

    # open stream
    stream = p.open(format=format, channels=args.channels, rate=args.rate, input=True, frames_per_buffer=args.chunk)


    # record for the given amount of time
    print("Start recording")

    frames = []
    for i in range(0,int(args.rate / args.chunk * args.seconds)):
        data = stream.read(args.chunk)
    frames.append(data)

    print("End recording")

    # stop the stream and close it
    stream.stop_stream()
    stream.close()

    # terminate pyaudio's object
    p.terminate()

    # set the filename
    if args.name == None:
        # concatenate the values
        FILENAME = "{}_{}Hz_{}s.wav".format(args.format, args.rate, args.seconds)


    # overwrite the file if it is already present
    # - workaround : delete it
    if FILENAME in os.listdir():
        os.remove(FILENAME)


    # setup the final audio file and save it
    wf = wave.open(FILENAME, 'wb')
    wf.setnchannels(args.channels)
    wf.setsampwidth(p.get_sample_size(format))
    wf.setframerate(args.rate)
    wf.writeframes(b''.join(frames))

# close write_file's object
    wf.close()

    print("File salved")


def main():

    parser = ArgumentParser()

    parser.add_argument('--chunk', type=int, default=1024, help='Set number of chunks')
    parser.add_argument('--format', type=str, default='Int16', help='Set the format of the audio track [Int8,Int16,Int32]')
    parser.add_argument('--channels', type=int, default=2, help='Set the number of channels')
    parser.add_argument('--seconds', type=int, default=3, help='Set the length of the recording (seconds)')
    parser.add_argument('--rate', type=int, default=44100, help='Set the rate')
    parser.add_argument('--name', type=str, default=None, help='Set the name of the audio track')
    parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")
    
    args = parser.parse_args()

    record_audio(args)

if __name__ == '__main__':
    main()



# Lab 2

## Ex 1 

In [4]:
from scipy.io import wavfile
from scipy import signal
import datetime
import numpy as np
import os

a = datetime.datetime.now()
rate, audio = wavfile.read('audio2.wav')
audio = signal.resample_poly(audio, 1, 16000)

b = datetime.datetime.now()

delta = b-a

print("The total time is:", (delta.total_seconds() * 1000)) 

audio = audio.astype(np.int16)
samplerate = 48000; fs = 16000
t = np.linspace(0., 1., samplerate)
amplitude = np.iinfo(np.int16).max
data = amplitude * np.sin(2. * np.pi * fs * t)
wavfile.write("ex.wav",48000,data.astype(np.int16))
print(os.path.getsize("audio2.wav"))
print(os.path.getsize("ex.wav"))

The total time is: 1206.8890000000001
889422
96044


## Ex.2

In [5]:
import tensorflow as tf

audio = tf.io.read_file('ex.wav')

tf_audio, rate = tf.audio.decode_wav(audio,1)
tf_audio = tf.squeeze(tf_audio,1)

stft = tf.signal.stft(tf_audio,frame_length=48,frame_step=96,fft_length=48)
a = datetime.datetime.now()

spectrogram = tf.abs(stft)

b = datetime.datetime.now()

delta = b-a
print("The total time is:", (delta.total_seconds() * 1000)) 

The total time is: 1.827


In [6]:
print(spectrogram)

tf.Tensor(
[[1.09202214e-04 3.51020244e-05 6.01649153e-05 ... 5.81319591e-06
  3.05315079e-05 5.37591768e-06]
 [1.36416274e-04 5.45332696e-05 5.59677537e-05 ... 1.73576609e-05
  1.64943012e-05 4.38441421e-05]
 [1.21128134e-04 5.08614503e-05 3.75189920e-05 ... 3.15201432e-05
  2.57155643e-05 1.65582169e-05]
 ...
 [1.06530577e-04 8.01821443e-05 5.29719327e-05 ... 4.11437250e-05
  6.23723754e-05 8.23469145e-06]
 [1.28837404e-04 5.69865297e-05 5.38902459e-05 ... 6.55624390e-05
  2.95024256e-05 1.17306845e-05]
 [1.16351213e-04 5.20606227e-05 1.28945940e-05 ... 1.00783782e-05
  5.09405618e-05 3.65012202e-05]], shape=(500, 25), dtype=float32)


In [7]:
audio_str = tf.io.serialize_tensor(tf_audio)
tf.io.write_file('audio_str',audio_str,name = None)

In [8]:
os.path.getsize('audio_str')

192014

In [9]:
rate, audio = wavfile.read('no_01.wav')
audio = signal.resample_poly(audio, 1, 3)
audio = audio.astype(np.int16)
wavfile.write("no_01_res.wav",16000,audio)



In [10]:
audio = tf.io.read_file('no_01_res.wav')

tf_audio, rate = tf.audio.decode_wav(audio,1)
tf_audio = tf.squeeze(tf_audio,1)

stft = tf.signal.stft(tf_audio,frame_length=48,frame_step=960,fft_length=49)

spectrogram = tf.abs(stft)



In [11]:
#Manage the spectrogram as image

image = tf.transpose(spectrogram)
image = tf.expand_dims(image, -1)
image = tf.math.log(image + 1.e-6)

#Min max normalization
min_ = tf.reduce_min(image)
max_ = tf.reduce_max(image)
image = (image - min_) / (max_ - min_)
image = image * 255.

image = tf.cast(image, tf.uint8)

In [14]:
img = tf.io.encode_png(image, name=None)

In [15]:
tf.io.write_file('spec.png',img,name = None)

## Ex.3 

In [19]:
audio_str = tf.io.parse_tensor(audio_str,tf.float32, name=None)

In [ ]:
#Rivedere i parametri, studiare letteratura
num_spectrogram_bins = spectrogram.shape[-1] 
linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(#16000)
mel_spectrogram = tf.tensordot(spectrogram, linear_to_mel_weight_matrix, 1) 
mel_spectrogram.set_shape(spectrogram.shape[:-1].concatenate(linear_to_mel_weight_matrix.shape[-1:])) 
log_mel_spectrogram = tf.math.log(mel_spectrogram + 1e-6)

In [ ]:
mfccs = tf.signal.mfccs_from_log_mel_spectrograms( log_mel_spectrogram)[..., :10]

In [ ]:
tf.io.write_file('mfcss',mfccs,name = None)
os.path.getsize('mfccs')

## Ex.4

In [25]:
img_prova = audio = tf.io.read_file('img_prova.jpg')

In [26]:
tf_img = tf.io.decode_jpeg(img_prova)

In [28]:
tf_img_cropped = tf.image.crop_to_bounding_box(tf_img,0,0,168,168)

In [30]:
a = datetime.datetime.now()

tf_img_resized = tf.image.resize (tf_img,[224,224])

b = datetime.datetime.now()
delta = b-a
print(delta)


0:00:00.011153


In [31]:
img_final = tf.cast(tf_img_resized, tf.uint8)

In [32]:
img_final = tf.io.encode_png(img_final, name=None)

In [33]:
tf.io.write_file('img_final.png',img,name = None)